# Truncate NTFA modes

Felix Fritzen <fritzen@simtech.uni-stuttgart.de>,
Julius Herb <julius.herb@mib.uni-stuttgart.de>,
Shadi Sharba <shadi.sharba@isc.fraunhofer.de>

University of Stuttgart, Institute of Applied Mechanics, Chair for Data Analytics in Engineering

> **Funding acknowledgment**
> The IGF-Project no.: 21.079 N / DVS-No.: 06.3341 of the
> “Forschungsvereinigung Schweißen und verwandte Verfahren e.V.” of the
> German Welding Society (DVS), Aachener Str. 172, 40223 Düsseldorf, Germany,
> was funded by the Federal Ministry for Economic Affairs and Climate Action (BMWK)
> via the German Federation of Industrial Research Associations (AiF) in accordance
> with the policy to support the Industrial Collective Research (IGF)
> on the orders of the German Bundestag.
>
> Felix Fritzen is funded by the German Research Foundation (DFG) --
> 390740016 (EXC-2075); 406068690 (FR2702/8-1); 517847245 (FR2702/10-1).

## Imports

In [1]:
import h5py
import numpy as np
import os

from material_parameters import *

data_path = "../data"

In [2]:
fname = os.path.join(
    data_path, "validation", "simple_3d_rve_B1-B6_16x16x16_10samples_fix.h5"
)
basename = "/ms_9p/dset0_ntfa/"

F = h5py.File(fname, "r")
A_bar = np.array(F[basename + "A_bar"])
A0 = np.array(F[basename + "A0"])
A1 = np.array(F[basename + "A1"])
C0 = np.array(F[basename + "C0"])
C1 = np.array(F[basename + "C1"])
C_bar = np.array(F[basename + "C_bar"])
D_theta = np.array(F[basename + "D_theta"])
D_xi = np.array(F[basename + "D_xi"])
tau_theta = np.array(F[basename + "tau_theta"])
tau_xi = np.array(F[basename + "tau_xi"])
temperatures = np.array(F[basename + "temperatures"])
vol_frac = np.array(F["ms_9p/dset0_ntfa"].attrs["combo_volume_fraction"])
# vol_frac = np.zeros(2)
# vol_frac[1] = np.mean(F["ms_1p/dset0_sim/mat_id"])
# vol_frac[0] = 1. - vol_frac[1]
# print(vol_frac)
mu = np.array(F["/ms_9p/dset0_sim/plastic_modes"])
mu_eff = np.linalg.norm(mu, axis=2).sum(axis=0)
# print(mu_eff/vol_frac[0])
print(np.mean(mu[:, :12, :], axis=0))
F.close()

[[-4.81957192e-03 -1.87528725e-03  6.69485918e-03 -2.51804015e-01
  -2.52646652e-01 -2.52964984e-01]
 [ 1.45886310e-03 -6.41553815e-04 -8.17309288e-04 -1.99404954e-01
  -1.58046155e-01  3.56296561e-01]
 [-8.78852426e-05  1.26334860e-03 -1.17546336e-03 -2.99336425e-01
   3.22711105e-01 -2.43784261e-02]
 [-2.80818244e-01 -9.75181120e-02  3.78336356e-01  4.02089368e-03
   6.01847530e-03  6.82159257e-03]
 [ 2.75147501e-01 -3.81194901e-01  1.06047400e-01 -2.82880727e-04
   1.71243014e-03 -1.01420933e-03]
 [-4.12447345e-02 -1.57588433e-02  5.70035778e-02 -1.04866728e-02
  -1.03093058e-02 -1.02459260e-02]
 [-7.86225341e-04 -1.96939329e-03  2.75561863e-03  1.27961341e-01
  -8.91150551e-02 -3.90480145e-02]
 [-2.50104340e-03  1.99375768e-03  5.07285721e-04  2.86961002e-02
   9.63577684e-02 -1.25873730e-01]
 [-8.39072811e-03 -3.99626799e-03  1.23869961e-02  1.00012553e-01
   1.00066432e-01  9.95045543e-02]
 [-4.14913018e-03 -1.52616497e-01  1.56765628e-01 -3.64521611e-03
  -6.63791917e-03 -5.1728

In [3]:
Ntemp = temperatures.size
Nmax = 24
# truncate and change order of the data
# print(A_bar.shape)
# phase average stress localization
A0 = np.transpose(A0[:, : Nmax + 7, :], axes=[2, 0, 1])
A1 = np.transpose(A1[:, : Nmax + 7, :], axes=[2, 0, 1])

In [4]:
sig_ph = np.zeros((2,) + A0.shape)
sig_ph[0] = A0
sig_ph[1] = A1

In [5]:
# phase stiffness tensors at the different temperatures
C0 = np.transpose(C0, axes=[2, 0, 1])
C1 = np.transpose(C1, axes=[2, 0, 1])

In [6]:
C_ph = np.zeros((2,) + C0.shape)
C_ph[0] = C0
C_ph[1] = C1

In [7]:
eps_th_ph = np.zeros((2, Ntemp, 6))
eps_th_ph[0, :, :] = eps_th_cu(temperatures)[:, None] * I2[None, :]
eps_th_ph[1, :, :] = eps_th_wsc(temperatures)[:, None] * I2[None, :]

In [8]:
A_bar = -np.transpose(A_bar[:, :Nmax, :], axes=[2, 1, 0])
# print(A_bar.shape)
D_xi = -np.transpose(D_xi[:Nmax, :Nmax, :], axes=[2, 0, 1])
# print(np.linalg.eigvals(D_xi[0,:3,:3]))
tau_xi = -np.transpose(tau_xi[:Nmax, :], axes=[1, 0])
tau_theta = np.transpose(tau_theta)
C_bar = np.transpose(C_bar, axes=[2, 0, 1])
A = A_bar[0]
XX = A.T @ np.linalg.pinv(A @ A.T) @ A
print(XX)

[[ 1.00000000e+00 -1.83013327e-16 -2.70616862e-16  0.00000000e+00
   2.91433544e-16 -3.07046055e-16]
 [-3.93782229e-16  1.00000000e+00 -2.64545330e-16 -1.47451495e-16
   1.64798730e-16 -8.41340886e-17]
 [ 1.43982049e-16  4.88758339e-16  1.00000000e+00 -3.41740525e-16
   6.34041430e-16 -2.46330734e-16]
 [ 2.76037873e-16  5.25079112e-16  1.72821826e-16  1.00000000e+00
  -2.83193607e-16  2.42427606e-16]
 [-2.25514052e-16 -5.44215280e-16 -2.50017021e-16 -1.92987987e-17
   1.00000000e+00 -1.72604986e-16]
 [-4.94396191e-17  1.68051337e-17  1.25984292e-16  2.04697370e-16
   2.87096735e-16  1.00000000e+00]]


In [9]:
# print(( vol_frac[1]*A1[0,:,:7]+vol_frac[0]*A0[0,:,:7]) )
x = C_bar[0, :, :].flatten()
x0 = A0[0, :, :6].flatten()
x1 = A1[0, :, :6].flatten()

In [10]:
alpha = np.dot(x - x1, x0 - x1) / np.dot(x0 - x1, x0 - x1)
# print(alpha, vol_frac)
alpha = 37 / 64
print("c_Bar:")
print(C_bar[0, :, :])

c_Bar:
[[2.83544622e+08 1.30812412e+08 1.30812412e+08 1.24569387e+05
  1.24569387e+05 7.44890553e+04]
 [1.30812412e+08 2.83544622e+08 1.30812412e+08 1.24569387e+05
  7.44890553e+04 1.24569387e+05]
 [1.30812412e+08 1.30812412e+08 2.83544622e+08 7.44890553e+04
  1.24569387e+05 1.24569387e+05]
 [1.24569387e+05 1.24569387e+05 7.44890553e+04 1.55475051e+08
  1.67818031e+05 1.67818031e+05]
 [1.24569387e+05 7.44890553e+04 1.24569387e+05 1.67818031e+05
  1.55475051e+08 1.67818031e+05]
 [7.44890553e+04 1.24569387e+05 1.24569387e+05 1.67818031e+05
  1.67818031e+05 1.55475051e+08]]


In [11]:
xx = alpha * x0 + (1 - alpha) * x1
# print(xx.reshape((6,6))-C_bar[0,:,:])
print("error:", xx / np.linalg.norm(x))

error: [4.44244702e-01 2.03578631e-01 2.03578631e-01 2.26026406e-04
 2.26026406e-04 1.38090640e-04 2.03578631e-01 4.44244702e-01
 2.03578631e-01 2.26026406e-04 1.38090640e-04 2.26026406e-04
 2.03578631e-01 2.03578631e-01 4.44244702e-01 1.38090640e-04
 2.26026406e-04 2.26026406e-04 2.18730561e-04 2.18730561e-04
 1.30794698e-04 2.45482202e-01 2.94670288e-04 2.94670288e-04
 2.18730561e-04 1.30794698e-04 2.18730561e-04 2.94670288e-04
 2.45482202e-01 2.94670288e-04 1.30794698e-04 2.18730561e-04
 2.18730561e-04 2.94670288e-04 2.94670288e-04 2.45482202e-01]


In [12]:
# print((C_bar[0,:,:]), tau_theta[0,:])
fname = f"data_new/ms9p_fix_ntfa16_B1-6_10s_N{Nmax}.h5"
F = h5py.File(fname, "w")
F.create_dataset("SIG_phases", data=sig_ph)
F.create_dataset("C_phases", data=C_ph)
F.create_dataset("eps_th_phases", data=eps_th_ph)
F.create_dataset("A_bar", data=A_bar)
F.create_dataset("v_frac", data=vol_frac)
F.create_dataset("C_bar", data=C_bar)
F.create_dataset("D_xi", data=D_xi)
F.create_dataset("D_theta", data=D_theta)
F.create_dataset("tau_xi", data=tau_xi)
F.create_dataset("tau_theta", data=tau_theta)
F.create_dataset("temperatures", data=temperatures)
F.close()
eps_th_500 = np.linalg.solve(C_bar[20, :, :], tau_theta[20, :])
print(eps_th_500)
print(tau_theta)

FileNotFoundError: [Errno 2] Unable to synchronously create file (unable to open file: name = 'data_new/ms9p_fix_ntfa16_B1-6_10s_N24.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)